# Create COOP and CORP NRs
This notebook contains snippets to create new NRs which can then be used manually or via existing notebooks to create a company.

## Setup environment
Before running following environment setup snippets, ensure environment variables found in `default-bcr-business-setup-TEST` notebook contain the correct values.

In [ ]:
%run /workspaces/lear/tests/data/default-bcr-business-setup-TEST.ipynb

In [ ]:
%run /workspaces/lear/tests/data/common/utility.ipynb
%run /workspaces/lear/tests/data/common/namex_api_utils_test_payload_data.ipynb
%run /workspaces/lear/tests/data/common/namex_api_utils.ipynb

import os
import re
import json

In [ ]:
entity_type = 'BC' # CP, BC
email = '<placeholder_email_address>'
verify_ssl=True # set to False if using proxy to debug requests

In [ ]:
namex_token = get_namex_token(verify_ssl)
assert namex_token
# namex_token

### Create NR

In [ ]:
dt_str = datetime.now().strftime("%Y.%m.%d %H.%M.%S")
print(f'dt: {dt_str}')

company_name = f'XYZ {dt_str} Test'
print(f'company_name: {company_name}')

create_nr_json = factory_create_nr_payload()
create_nr_json['applicants'][0]['emailAddress'] = email
create_nr_json['names'][0]['name'] = company_name
create_nr_json['names'][0]['name_type_cd'] = 'CO'
create_nr_json['names'][0]['designation'] = 'CORP.' if entity_type == 'BC' else None
create_nr_json['entity_type_cd'] = entity_type

r = create_nr(create_nr_json, verify_ssl)
response_dict = json.loads(r.text)
nr_id = response_dict.get('id')
nrl = response_dict.get('nrNum')
nr_name_id = response_dict['names'][0]['id']
nr_name = response_dict['names'][0]['name']
name_type_cd = response_dict['names'][0]['name_type_cd']

print (f'nr_id: {nr_id}')
print (f'nrl: {nrl}')


### Create Payment

In [ ]:
create_payment_json = factory_create_nr_payment_payload()
create_payment_json['filingInfo']['filingTypes'][0]['filingTypeCode'] = 'NM620'
r = create_nr_payment(nr_id, nrl, email, create_payment_json, verify_ssl)
r.status_code, r.text
response_dict = json.loads(r.text)
nr_num = response_dict.get('nrNum')
payment_id = response_dict['payment']['id']
print (f'payment_id: {payment_id}')
payment_token = response_dict['payment']['payment_token']
print (f'payment_token: {payment_token}')
print (f'nr_num: {nr_num}')
invoice_id = response_dict['sbcPayment']['references'][0]['invoiceNumber']
print(f'invoice_id: {invoice_id}')

## Create/Update payment transaction
Note: Use the pay_sys_url provided after executing the snippet below to complete the payment before continuing the notebook snippets below.  After paying using the test credit card, you should be redirected a name request page with a dialog indicating that your NR encountered an error and please do not try submitting again.  This is fine.  You should be able to continue with running the snippets below at this point.

In [ ]:
r = create_nr_payment_transaction(nr_id, payment_id, payment_token, verify_ssl)
r.status_code, r.text
response_dict = json.loads(r.text)
transaction_id = response_dict.get('id')
pay_sys_url = response_dict.get('paySystemUrl')
print('Use following link to make payment in browser.  Continue with snippets below after making payment.')
print(f'pay_sys_url: {pay_sys_url}')

## Update NR status to IN PROGRESS

In [ ]:
update_nr_state_in_progress_json = {
    'state': 'INPROGRESS'
}

r = update_nr_state(namex_token, nr_num, update_nr_state_in_progress_json, verify_ssl)
r.status_code, r.text

## Update NR name state to APPROVED

In [ ]:
payload = factory_update_nr_name_payload()
payload['choice'] = 1
payload['id'] = nr_name_id
payload['name'] = nr_name
payload['name_type_cd'] = name_type_cd
payload['state'] = 'APPROVED'

r = update_nr_name(namex_token, nr_num, payload, verify_ssl)
r.status_code, r.text

## Update NR state to APPROVED

In [ ]:
update_nr_state_approved_json = {
    'state': 'APPROVED'
}

r = update_nr_state(namex_token, nr_num, update_nr_state_approved_json, verify_ssl)
r.status_code, r.text